<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_4_docx_legal_analyzer_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧾 Day 4 — Office Administrator: Legal Doc Analyzer (DOCX edition)

Этот ноутбук поможет вам создать ассистента, который анализирует юридические документы в формате `.docx` (договоры, политики, инструкции) и отвечает на вопросы с указанием источников.

📌 Сегодня вы научитесь:
- Устанавливать зависимости для работы с DOCX
- Загружать документы с помощью `UnstructuredReader`
- Создавать QueryEngine для вопросов
- Расширим: сравнение версий договоров


In [1]:

# 🛠️ Установка необходимых библиотек
!pip install "unstructured[all-docs]" llama-index --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 41.3 MB/s eta 0:00:00
  


⚠️ **Важно:** После установки:
- Перезапустите среду выполнения (`Среда выполнения` → `Перезапустить`)
- Затем выполните следующую ячейку


In [6]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [7]:

from pathlib import Path
from llama_index.core import VectorStoreIndex
from llama_index.readers.file import UnstructuredReader

Path("data").mkdir(exist_ok=True)

reader = UnstructuredReader()
docs = reader.load_data(file=Path("data/Договор_с_банковскими_реквизитами.docx"))
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine()


In [8]:
def ask(query):
    response = query_engine.query(query)
    print("🔹 Ответ:")
    print(response.response)
    print("\n🔸 Источники:")
    for i, node in enumerate(response.source_nodes):
        print(f"\nИсточник {i+1}:")
        print(node.get_text().strip())


In [9]:

ask("Какие обязанности у арендатора?")


🔹 Ответ:
Использовать Квартиру только для проживания, содержать имущество и сети в исправном состоянии, не передавать права третьим лицам без согласия Арендодателя.

🔸 Источники:

Источник 1:
• Расчётный счёт: 40702810400000001234

4.3. Гарантийный депозит в размере 85 000 (восемьдесят пять тысяч) рублей вносится Арендатором единовременно до подписания передаточного акта и не позднее даты въезда.

4.4. Депозит хранится на условном счёте Арендодателя и возвращается в течение 10 рабочих дней после окончания действия Договора за вычетом удержаний.

5. ПОРЯДОК ПЕРЕДАЧИ ИМУЩЕСТВА

5.1. Передача Квартиры и имущества оформляется передаточным актом (Приложение №2), подписываемым Сторонами в день начала аренды.

5.2. С момента подписания акта и оплаты депозита Арендатор считается принявшим Квартиру и имущество в надлежащем состоянии.

6. ПРАВА И ОБЯЗАННОСТИ СТОРОН

6.1. Обязанности Арендодателя:

• Передать помещение в пригодном для проживания состоянии.

• Обеспечить работу всех инженерных сис


## 🔁 Расширение: сравнение версий договоров

Если у вас есть `v1.docx` и `v2.docx`, можно проанализировать оба:


In [10]:

reader = UnstructuredReader()
docs_v1 = reader.load_data(file=Path("/content/data/Договор_с_банковскими_реквизитами.docx"))
docs_v2 = reader.load_data(file=Path("/content/data/Договор_с_банковскими_реквизитами_1.docx"))

index_v1 = VectorStoreIndex.from_documents(docs_v1)
index_v2 = VectorStoreIndex.from_documents(docs_v2)

qe_v1 = index_v1.as_query_engine()
qe_v2 = index_v2.as_query_engine()

print("📄 Версия 1:")
print(qe_v1.query("Какая ежемесячная арендная плата?").response)

print("\n📄 Версия 2:")
print(qe_v2.query("Какая ежемесячная арендная плата?").response)


📄 Версия 1:
Ежемесячная арендная плата составляет 85 000 (восемьдесят пять тысяч) рублей.

📄 Версия 2:
95 000 (девяноста пять тысяч) рублей.



## ✅ Чеклист

- [ ] Установлены библиотеки и перезапущен runtime  
- [ ] Загружен DOCX документ  
- [ ] Запросы работают, ответы с цитатами  
- [ ] (опц.) Сравнение двух версий договора  
